In this qucikstart we'll see how to :
- get setup with LangChain, LangSmith and LangServe
- USe the most basic and common components of Langchain: Prompt templates , models and output parser.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")
print(llm)

In [ ]:
## Input and get resposne from LLM
result = llm.invoke("What is generative AI?")
result

In [ ]:
### Chatpromt Template
from langchain_core.promts import ChatPromtTemplate
prompt = ChatPromtTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answer based on the question"),
        ("user","{input}")
    ]
)
prompt

In [ ]:
## Chain
chain = prompt|llm
response = chain.invoke({"input":"can you tell me about Langsmith?"})
response

In [ ]:
## type = AIMessage
type(response)

In [ ]:
## stroutput Parser
## convert the output content to string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser  # first go to prompt then llm then output_parser

response = chain.invoke({"input":"Can you tell me about Langsmith?"})
response

#### Simple GEN AI 

In [ ]:
## Data injection - From the webiste we need to scrap tha data 
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
docs = loader.load()
docs

In [ ]:
# Document splitter, convert the docment to the some chunk
# Because evey llm model has restriction with reapect to the context size


In [ ]:
# After converting to the chunks , convert into the vectors (Vector Embedding)
# Store to the Vector Store
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_Splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, cunk_overlap = 200)
documents = text_Splitter.split_documents(docs)
documents

#### vector Embedding using the Hugging Face

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # Load all the environment varaibles
os.environ["HUGGING_FACE_TOKEN"] = os.getenv("HUGGING_FACE_TOKEN")


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_community.vectorstores import FAISS
VECTORSTOREDB = FAISS.from_documents(documents,embedding)

In [ ]:
## Query From a vector db
query = "LangSmith has two usage limits: total traces and extended"
result = VECTORSTOREDB.similarity_search(query)

In [ ]:
result[0].page_content

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
print(llm)

In [ ]:
## Retrieval Chain, Documents Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
promt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided conext:
    <context>
    {context}
    </context>
    """
)
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_context = "LangSmith has two usage limits: total traces and extended traces")]
})

##
However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevent doucmnets and pass those in for a given question.

In [ ]:
### Retriever --> It as an interface which is used to get the data from the vector db
## convert the vectordb to the retriever. use retriever to get the result from the vector store


retriever = VECTORSTOREDB.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever)

In [ ]:
## Get the response from the LLM
response = retriever_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended"
})
response['answer']

In [ ]:
response['context']

In [ ]:
response

# Simple Gen AI App using Ollama
